In [1]:

import cyvcf2
import tsinfer
import pandas as pd
import json
import numpy as np
import sys
import os

In [2]:
tsinfer.__version__

'0.3.0'

In [31]:
chromosome = 1
vcfFile = "/home/jana/Documents/1Projects/HoneybeeDemo/Tsinfer/Chr1_ancestral.vcf.gz"
meta = "/home/jana/Documents/1Projects/HoneybeeDemo/Tsinfer/SampleMetaData.csv"
sampleFile = "Chr1.samples"
ploidy = 1
chrLength = {1:1000000, 2:2000000}

In [32]:
print("Chromosome is " + str(chromosome))

Chromosome is 1


In [33]:
def add_individuals(vcf, samples, populations, metaData, ploidy):
    """
    The function to store the information about which nodes pertain to which sample
    Could be modified in case of a different ploidy
    :param vcf: Vcf file of your samples
    :param samples:
    :param populations:
    :param metaData: meta data file with ID, Pop, Subpop, and time
    :return:
    """
    for name, population in zip(vcf.samples, populations):
        samples.add_individual(ploidy=ploidy, metadata={"name": name}, population=population,time=list(metaData.Time[metaData.ID == name])[0])

def add_sites(vcf, samples, ploidy):
    """
    Read the sites in the vcf and add them to the samples object, reordering the
    alleles to put the ancestral allele first, if it is available.
    """
    pos = 0
    for variant in vcf:  # Loop over variants, each assumed at a unique site
        if pos == variant.POS:
            raise ValueError("Duplicate positions for variant at position", pos)
        else:
            pos = variant.POS
        if ploidy > 1:
            if any([not phased for _, _, phased in variant.genotypes]):
                raise ValueError("Unphased genotypes for variant at position", pos)
        alleles = [variant.REF] + variant.ALT
        ancestral = variant.INFO.get("AA", variant.REF)
        
        try:
            ancestral_allele = alleles.index(ancestral[0])
            #print(pos, ancestral, ancestral_allele)
        except:
            ancestral_allele = MISSING_DATA


        genotypes = [g for row in variant.genotypes for g in row[0:ploidy]]
        samples.add_site(position=pos,
                             genotypes=genotypes,
                             alleles=alleles,
                             ancestral_allele=ancestral_allele)


# This function checks whether there is only one contig in the vcf - this may not be the case!
def chromosome_length(vcf, chromosome):
    """
    This is a modified function to read in the sequence length
    :param vcf: vcffile
    :return: the length of the sequence (genome or chromosme - depending what the vcf is for)
    """
    #assert len(vcf.seqlens) == 1
    return vcf.seqlens[int(chromosome)-1]


def add_populations(vcf, samples, metaData):
    """
    This is a modified add populations function for drones from David Wragg data
    Add tsinfer Population objects for drone subspecies, stored in metaPop object, and return a list od IDs correposning to the VCF samples
    Input: vcf = cyvcf2 VCF object, samples is tsinfer SampleData and metaData is a pandas dataframe with id in ID column and populations in Type column
    pops = dictinary holding name and additional information about populations
    Return: A list of population indexes
    """
    pop_lookup = {}
    metaPop = metaData[['Pop', 'SubPop']].drop_duplicates().dropna(axis = 0, how = 'all')
    sample_subpop = [list(metaData.SubPop[metaData.ID == x]) for x in vcf.samples]
    for subpop, pop in zip(metaPop.SubPop, metaPop.Pop):
        pop_lookup[subpop] = samples.add_population(metadata={"pop": pop, "subpop": subpop})
    return [pop_lookup[subpop[0]] for subpop in sample_subpop]

In [34]:
metaFile = pd.read_csv(meta)
metaFile.head()
len(metaFile)


692

In [35]:
vcfD = cyvcf2.VCF(vcfFile, strict_gt=True)

In [36]:
vcf = vcfD
metaData = metaFile

In [37]:
with tsinfer.SampleData(path=sampleFile, 
                        sequence_length=chrLength[chromosome]) as samples:
   populations = add_populations(vcf = vcfD, samples = samples, metaData = metaFile)
   print("populations determined")
   add_individuals(vcf = vcfD, samples = samples, populations = populations, metaData = metaFile, ploidy = ploidy)
   print("Inds added")
   add_sites(vcf = vcfD, samples = samples, ploidy = ploidy)

populations determined
Inds added


In [38]:
print(
   "Sample file created for {} samples ".format(samples.num_samples)
   + "({} individuals) ".format(samples.num_individuals)
   + "with {} variable sites.".format(samples.num_sites)
)

Sample file created for 692 samples (692 individuals) with 20 variable sites.
